In [1]:
# Setup display width, load packages, import symbols
ENV["COLUMNS"]=72
using Pkg; for p in ("Knet","Plots","PyCall","JSON","JLD2","FileIO","ImageMagick"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Base.Iterators: flatten
using Statistics: mean
using Plots;
using FileIO;
using MAT;
using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, Param,progress, sgd,dropout, relu,
Data,sigm, xavier,gpu,softmax
import Knet:param,param0,Data
using Images;
using PyCall,JSON,JLD2,Random
@pyimport numpy as np 

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:400
└ @ PyCall /home/ege/.julia/packages/PyCall/RQjD7/src/PyCall.jl:400


In [2]:
include("minibatch_updated.jl")

In [3]:
struct Chain
    layers
    Chain(layers...)=new(layers)
end
(c::Chain)(x) = (for layer in c.layers; x=layer(x) end;x)#if printsize==1 println(size(x)) end;end; x) #layerların çıktılarını hesaplayıp son layerın çıktısını veriyor
(c::Chain)(x,y)=nll(c(x),y) #loss
(c::Chain)(d::Data)=mean(c(x,y) for (x,y) in d) #batch için mean loss hesapla

In [11]:
struct Ensemble model1;model2;alpha;
    Ensemble(model1,model2,alpha)=new(model1,model2,alpha)
end
(e::Ensemble)(x1,x2)=e.model1(x1)+e.model2(x2)#e.model1(x1)*alpha+e.model2(x2)*(1. - alpha)
(e::Ensemble)(x1,x2,y)=nll(e(x1,x2),y) #loss
(e::Ensemble)(d::Data3d)=mean(e(x1,x2,y) for (x1,x2,y) in d)

In [5]:
struct Dense; w;b;func;
    Dense(w,b)=new(Param(w),Param(b),relu)
    Dense(w,b,func)=new(Param(w),Param(b),func)
    Dense(x::Int,y::Int,func)=new(param(y,x),param0(y),func)
    end;
function (den::Dense)(x) 
    matrix_multip=den.w*mat(x).+den.b
    if den.func==relu
        return den.func.(matrix_multip)
    else
        return den.func(matrix_multip)
    end
end

In [46]:
# For running experiments
function trainresults2(model,dtrn,ep,optimizer,learning_r)
    train_iter(n,itr) = (x for (i,x) in enumerate(itr) if i % n == 1)
                
        r = ((model(dtrn))
             for x in train_iter(length(dtrn), progress(optimizer(model,repeat(dtrn,ep),lr=learning_r))))
        r = collect(Float32,flatten(r))          
    return r
end

trainresults2 (generic function with 1 method)

In [61]:
dense_1=Chain(Dense(2,20,relu),Dense(20,2,relu));
dense_2=Chain(Dense(2,10,relu),Dense(10,2,relu));
ensemble=Ensemble(dense_1,dense_2,0.5);

In [63]:
x1_data=convert(Array{Float32},rand(2))
x2_data=convert(Array{Float32},rand(2))
for i in 1:7999
    x1_data=cat(x1_data,convert(Array{Float32},rand(2)),dims=2)
    x2_data=cat(x2_data,convert(Array{Float32},rand(2)),dims=2)
end
y_data=[]
for i in 1:8000
    if sum(x1_data[:,i])>sum(x2_data[:,i])
        push!(y_data,1)
    else
        push!(y_data,2)
    end
end
little_batch=minibatch_3d(x1_data,x2_data,y_data,100,ytype=Array{Int32});

In [64]:
results=trainresults2(ensemble,little_batch,20,sgd,0.1);

7.29e-02  100.00%┣██████████████████┫ 1600/1600 [00:01/00:01, 2369.51i/s]


In [82]:
correct=0
for i in 1:1000
    a=rand(2,1)
    b=rand(2,1) 
    label = sum(a) > sum(b) ? 1 : 2
    pred=ensemble(a,b)
    if findmax(pred)[2][1]==label
        correct+=1
    end
end
println(correct/1000)

0.993


In [90]:
a=rand(2,1)
b=rand(2,1) 
label = sum(a) > sum(b) ? 1 : 2
println(a," ",sum(a))
println(b," ",sum(b))
println(label)
pred=ensemble(a,b)
print(pred)

[0.868977; 0.528583] 1.3975604915545508
[0.941998; 0.160222] 1.102220167846082
1
[17.1357; 13.2894]

In [96]:
little_batch=minibatch_3d(x_b1,x_b2,y_b2,1,ytype=Array{Int32});

In [97]:
for (x1,x2,y) in little_batch
    println(x1," ",x2," ",y)
end

Float32[5.0; 8.0] Float32[7.0; 6.0] Int32[1]
Float32[7.0; 1.0] Float32[5.0; 5.0] Int32[2]
Float32[8.0; 8.0] Float32[6.0; 3.0] Int32[3]
Float32[10.0; 1.0] Float32[8.0; 8.0] Int32[4]
Float32[2.0; 6.0] Float32[6.0; 6.0] Int32[5]
Float32[4.0; 7.0] Float32[1.0; 5.0] Int32[6]
